# Import thư viện cần thiết

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

Đọc dữ liệu

In [2]:
train = pd.read_csv('../data/train_preprocessed.csv')
train.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target,rez_esc-missing
0,ID_279628684,190000.0,0,3,0,1,1,0,0.0,0,...,1849,1,100,0,1.000000,0.0,100.0,1849,4,False
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,4489,1,144,0,1.000000,64.0,144.0,4489,4,False
2,ID_68de51c94,0.0,0,8,0,1,1,0,0.0,0,...,8464,1,0,0,0.250000,64.0,121.0,8464,4,False
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,289,16,121,4,1.777778,1.0,121.0,289,4,False
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,1369,16,121,4,1.777778,1.0,121.0,1369,4,False


In [3]:
test = pd.read_csv('../data/test_preprocessed.csv')
test.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,rez_esc-missing
0,ID_2f6873615,0.0,0,5,0,1,1,0,0.0,1,...,0,16,9,0,1,2.25,0.25,272.25,16,False
1,ID_1c78846d2,0.0,0,5,0,1,1,0,0.0,1,...,256,1681,9,0,1,2.25,0.25,272.25,1681,False
2,ID_e5442cf6a,0.0,0,5,0,1,1,0,0.0,1,...,289,1681,9,0,1,2.25,0.25,272.25,1681,False
3,ID_a8db26a79,0.0,0,14,0,1,1,1,1.0,0,...,256,3481,1,256,0,1.00,0.00,256.00,3481,False
4,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,0,...,121,324,1,0,1,0.25,64.00,NaN,324,True


In [4]:
ntrain = train.shape[0]
ntest = test.shape[0]

all_data = pd.concat((train, test)).reset_index(drop=True)

# Xây dựng mô hình học máy

## Loại bỏ những thuộc tính dư thừa

In [5]:
all_data.set_index(['Id', 'idhogar'], inplace=True)

#### Kiểm tra các cột bị trùng lặp

In [6]:
temp = all_data.T.duplicated()
all_data[temp[temp==True].index]

,,hhsize,hogar_total,tipovivi5,agesq
Id,idhogar,,,,
ID_279628684,21eb7fcc1,1,1,0,1849
ID_f29eb3ddd,0e5d7a658,1,1,0,4489
ID_68de51c94,2c7317ea8,1,1,0,8464
ID_d671db89c,2b58d945f,4,4,0,289
ID_d56d6f5f5,2b58d945f,4,4,0,1369
...,...,...,...,...,...
ID_a065a7cad,3aa78c56b,6,6,0,100
ID_1a7c6953b,d237404b6,4,4,0,2916
ID_07dbb4be2,d237404b6,4,4,0,144


In [7]:
for col in all_data.columns:
    if all_data[col].equals(all_data['hhsize']):
        print(col)
print('------------------------')
for col in all_data.columns:
    if all_data[col].equals(all_data['hogar_total']):
        print(col)
print('------------------------')
for col in all_data.columns:
    if all_data[col].equals(all_data['tipovivi5']):
        print(col)
print('------------------------')
for col in all_data.columns:
    if all_data[col].equals(all_data['agesq']):
        print(col)

tamhog
hhsize
hogar_total
------------------------
tamhog
hhsize
hogar_total
------------------------
tipovivi4
tipovivi5
------------------------
SQBage
agesq


#### Theo kết quả ở trên, ta thấy rằng cột hhsize và hogar_total giống nhau, vì thế ta sẽ bỏ hết các cột trùng với 2 cột này và chỉ giữ lại 1 trong 2

In [8]:
all_data.drop(columns =['tamhog','hhsize'], inplace = True)

#### Sau đó tiến hành bỏ các cột trùng khác

In [9]:
all_data.drop(columns =['tipovivi5','SQBage'], inplace = True)

#### Loại bỏ các cột chỉ có 1 giá trị độc nhẩt

In [10]:
for column in all_data.columns:
    if all_data[column].nunique() == 1:
        all_data.drop(column, axis=1, inplace=True)

all_data

,,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,...,SQBescolari,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target,rez_esc-missing
Id,idhogar,,,,,,,,,,,,,,,,,,,,,
ID_279628684,21eb7fcc1,190000.0,0,3,0,1,1,0,0.0,0,1,...,100,1,100,0,1.000000,0.00,100.0000,1849,4.0,False
ID_f29eb3ddd,0e5d7a658,135000.0,0,4,0,1,1,1,1.0,0,1,...,144,1,144,0,1.000000,64.00,144.0000,4489,4.0,False
ID_68de51c94,2c7317ea8,0.0,0,8,0,1,1,0,0.0,0,0,...,121,1,0,0,0.250000,64.00,121.0000,8464,4.0,False
ID_d671db89c,2b58d945f,180000.0,0,5,0,1,1,1,1.0,0,2,...,81,16,121,4,1.777778,1.00,121.0000,289,4.0,False
ID_d56d6f5f5,2b58d945f,180000.0,0,5,0,1,1,1,1.0,0,2,...,121,16,121,4,1.777778,1.00,121.0000,1369,4.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_a065a7cad,3aa78c56b,0.0,1,2,1,1,1,0,0.0,0,2,...,9,36,25,4,36.000000,0.25,33.0625,100,NaN,False
ID_1a7c6953b,d237404b6,0.0,0,3,0,1,1,0,0.0,0,1,...,36,16,36,4,4.000000,1.00,36.0000,2916,NaN,False
ID_07dbb4be2,d237404b6,0.0,0,3,0,1,1,0,0.0,0,1,...,16,16,36,4,4.000000,1.00,36.0000,144,NaN,False


#### Chia dữ liệu 

In [11]:
train = all_data[:ntrain][:]
test = all_data[ntrain:][:]

## Xây dựng model

#### Import các thư viện cần thiết

In [12]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error,make_scorer

In [13]:
X = train.drop('Target', axis = 1)
Y = train['Target']

#### Fill các giá trị còn thiếu

In [14]:
imputer = SimpleImputer(missing_values=np.nan, strategy="median")
X = imputer.fit_transform(X)
# new_test_df = imputer.fit_transform(test.iloc[:, :-1])

#### Chia tập train và test cho model

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X ,Y, test_size=0.2)

#### Deploy model và tiến hành chạy thử 

In [16]:
knnModel = Pipeline([('model',KNeighborsRegressor())])
knnModel.fit(X_train,y_train)
predictions=knnModel.predict(X_test)

#### Hàm tính độ chính xác 

In [17]:
def accuracy(y, y_hat):
    return (y == y_hat).sum() / y.shape[0]

In [18]:
print(accuracy(y_test, predictions))

0.24332171893147503


#### Cải thiện model

In [19]:
knnModel = Pipeline([('scaler',StandardScaler()),('model',KNeighborsRegressor())])
knnModel.fit(X_train,y_train)
predictions=knnModel.predict(X_test)

In [20]:
print(accuracy(y_test, predictions))

0.3635307781649245


In [21]:
knnModel = Pipeline([('scaler',StandardScaler()),('model',KNeighborsRegressor())])
mod=GridSearchCV(estimator=knnModel,
                param_grid={'model__n_neighbors':np.arange(1,50)},
                scoring={'mse':make_scorer(accuracy)},
                refit='mse',
                cv=5)
mod.fit(X,Y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', KNeighborsRegressor())]),
             param_grid={'model__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])},
             refit='mse',
             scoring={'mse': make_scorer(accuracy, response_method='predict')})

In [22]:
mod.best_params_

{'model__n_neighbors': 1}

In [23]:
modified_knnModel = Pipeline([('scaler',StandardScaler()),('model',
                                KNeighborsRegressor(n_neighbors=mod.best_params_['model__n_neighbors']))])
modified_knnModel.fit(X_train,y_train)
knn_predictions=modified_knnModel.predict(X_test)

In [24]:
print(accuracy(y_test, knn_predictions))

0.8304297328687572
